<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLM_CRYPTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt -q

In [6]:
import os
import json
import time
import ccxt
from google import genai
from google.colab import userdata
from typing import Dict, Any, Union

# --- 1. Configuration and Initialization ---

# 🛑 CORRECTED: Explicitly assign the Kraken keys from userdata.get()
GEMINI_API_KEY = userdata.get('GEMINI')
KRAKEN_API_KEY = userdata.get('KRAKEN')
KRAKEN_SECRET = userdata.get('KRAKEN_SECRET')

# Corrected Gemini Initialization: Pass the key directly to the Client.
if GEMINI_API_KEY:
    # client is defined globally for use in the analysis function
    client = genai.Client(api_key=GEMINI_API_KEY)
    print("Gemini Client initialized successfully. Model: gemini-2.5-flash")
else:
    raise ValueError("ERROR: GEMINI API key not found in Colab secrets. Please set it.")

class CryptoAgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    TRADING_PAIR: str = "ETH/USD"
    ORDER_BOOK_DEPTH: int = 5    # Top 5 bids and asks

# --- 2. Data Retrieval Function (CCXT) ---

def fetch_kraken_order_book(symbol: str, limit: int) -> Union[Dict[str, Any], None]:
    """Fetches the top N bids and asks for a symbol from Kraken using CCXT."""
    print(f"\n--- 📈 Fetching Real-Time Data for {symbol} on Kraken ---")

    try:
        # CCXT initialization with the CORRECTLY assigned Kraken keys
        exchange = ccxt.kraken({
            'apiKey': KRAKEN_API_KEY,
            'secret': KRAKEN_SECRET,
            'enableRateLimit': True,
        })

        # Fetch the Order Book
        order_book = exchange.fetch_order_book(symbol, limit=limit)

        # Simplify the output for the LLM prompt, ensuring the timestamp is captured
        simplified_book = {
            'timestamp_utc': exchange.iso8601(order_book['timestamp']),
            'symbol': symbol,
            'bids_buyers': order_book['bids'], # [price, volume]
            'asks_sellers': order_book['asks'], # [price, volume]
            'current_mid_price': (order_book['bids'][0][0] + order_book['asks'][0][0]) / 2 if order_book['bids'] and order_book['asks'] else 'N/A'
        }

        print(f"Data fetched successfully at {simplified_book['timestamp_utc']}")
        return simplified_book

    except Exception as e:
        print(f"CCXT Error: Failed to fetch data. Ensure {symbol} is valid: {type(e).__name__}: {e}")
        return None

# --- 3. LLM Analysis Function (Gemini) ---

def analyze_market_depth_with_retry(market_data: Dict[str, Any], max_retries: int = 3) -> str:
    """Passes market data to Gemini 2.5 Flash for structured, actionable analysis with retries."""

    model = CryptoAgentConfig.LLM_MODEL_NAME
    data_string = json.dumps(market_data, indent=2)

    prompt = f"""
    You are a high-speed crypto trading analyst. Analyze the following real-time
    Kraken Order Book data for {CryptoAgentConfig.TRADING_PAIR}.

    Order Book Data:
    ---
    {data_string}
    ---

    Analyze the volume imbalance between 'bids_buyers' (buy pressure) and 'asks_sellers' (sell pressure).
    Respond ONLY in a structured JSON format with three key insights:

    1. **Imbalance:** State which side (Bids/Asks) has the heavier volume in the top {CryptoAgentConfig.ORDER_BOOK_DEPTH} layers.
    2. **Analysis:** Interpret the near-term price movement (e.g., likely to rally, minor pressure, consolidated).
    3. **Recommendation:** Provide a brief, neutral trading action (e.g., 'Wait for volume confirmation', 'Monitor for a break above X price', 'Consolidate position').

    Format ONLY as a JSON object:
    {{
      "imbalance_summary": "[Summary of volume difference]",
      "near_term_analysis": "[Price trend interpretation]",
      "actionable_insight": "[Recommendation]"
    }}
    """

    for attempt in range(max_retries):
        print(f"\n--- 🧠 Sending Data to Gemini LLM for Analysis (Attempt {attempt + 1}) ---")
        try:
            # Re-use the global 'client' object
            response = client.models.generate_content(
                model=model,
                contents=prompt,
                config={"response_mime_type": "application/json"}
            )
            return response.text # Success!

        except Exception as e:
            error_message = str(e)
            if '503 UNAVAILABLE' in error_message and attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"Server UNAVAILABLE (503). Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                # Log the final error if retries fail or if it's a different error
                print(f"Final error on attempt {attempt + 1}: {error_message}")
                return json.dumps({"error": f"Gemini API Error (Final): {error_message}"})

    return json.dumps({"error": "Failed to get a response after all retries."})


# --- 4. Main Execution ---

if __name__ == "__main__":

    # A. Fetch Real-Time Data
    market_data = fetch_kraken_order_book(
        CryptoAgentConfig.TRADING_PAIR,
        CryptoAgentConfig.ORDER_BOOK_DEPTH
    )

    if market_data and 'error' not in market_data:
        # B. Analyze Data with Retry Logic
        llm_output_json = analyze_market_depth_with_retry(market_data)

        # C. Display LLM's Final Structured Analysis
        print("\n--- 🤖 LLM's Final Structured Trading Analysis ---")
        try:
            final_analysis = json.loads(llm_output_json)
            print(json.dumps(final_analysis, indent=4))
        except json.JSONDecodeError:
            print("Failed to decode JSON from LLM. Raw output:")
            print(llm_output_json)
    elif market_data and 'error' in market_data:
        print(f"\nExecution terminated due to CCXT error: {market_data['error']}")
    else:
        print("\nExecution terminated due to failure in fetching market data.")

print("\nThis demo shows how LLMs can transform raw exchange data into actionable, structured trading insights.")

Gemini Client initialized successfully. Model: gemini-2.5-flash

--- 📈 Fetching Real-Time Data for ETH/USD on Kraken ---
Data fetched successfully at None

--- 🧠 Sending Data to Gemini LLM for Analysis (Attempt 1) ---

--- 🤖 LLM's Final Structured Trading Analysis ---
{
    "imbalance_summary": "Asks (sellers) have significantly heavier volume (114.78 ETH) in the top 5 layers compared to Bids (buyers) (2.901 ETH). There is a substantial sell wall directly above the current mid-price.",
    "near_term_analysis": "The strong sell pressure immediately above the mid-price suggests that ETH/USD will likely face significant resistance and could experience minor downward pressure or consolidation in the very near term. A rally appears difficult without clearing the large ask volume.",
    "actionable_insight": "Monitor for a reduction in ask-side volume or a clear break above 4014.14 USD with strong buying volume for potential upside. Currently, remain cautious and observe for further price a